In [1]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My\ Drive/nlp_final

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive/My Drive/nlp_final


In [0]:
import pandas as pd
import numpy as np

In [0]:
def convert_test(df):
    df1 = df.drop('RandomFifthSentenceQuiz2', axis=1)
    df2 = df.drop('RandomFifthSentenceQuiz1',axis=1)
    df1.rename(columns={'RandomFifthSentenceQuiz1':'Quiz'}, inplace=True)
    df2.rename(columns={'RandomFifthSentenceQuiz2':'Quiz'}, inplace=True)
    
    df1["Story"] = df1['InputSentence1'].str.cat(df1['InputSentence2'])
    df1["Story"] = df1['Story'].str.cat(df1['InputSentence3'])
    df1["Story"] = df1['Story'].str.cat(df1['InputSentence4'])
    df1 = df1.drop(columns=['InputSentence1','InputSentence2','InputSentence3','InputSentence4'])
    df1 = df1[['InputStoryid','Story','Quiz']]
    
    df2["Story"] = df2['InputSentence1'].str.cat(df2['InputSentence2'])
    df2["Story"] = df2['Story'].str.cat(df2['InputSentence3'])
    df2["Story"] = df2['Story'].str.cat(df2['InputSentence4'])
    df2 = df2.drop(columns=['InputSentence1','InputSentence2','InputSentence3','InputSentence4'])
    df2 = df2[['InputStoryid','Story','Quiz']]
    
    
    return df1,df2

In [0]:
df_val = pd.read_csv("data/ROC-Story-Cloze-Val.csv")

In [0]:
test1,test2 = convert_test(df_val)
test1.to_csv("tmp/test1/test.tsv",sep="\t",index=None)
test2.to_csv("tmp/test2/test.tsv",sep="\t",index=None)

In [0]:
ckpt_file = 'model/model11374.ckpt'

In [11]:

!python bin/story_classifier.py --task_name=STORY --do_predict=true --data_dir=tmp/test1 --vocab_file=bin/pre/vocab.txt --bert_config_file=bin/pre/bert_config.json --init_checkpoint=$ckpt_file --max_seq_length=60 --output_dir=tmp/res1





W1205 23:19:25.550996 140157866133376 module_wrapper.py:139] From bin/story_classifier.py:784: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.


W1205 23:19:25.551180 140157866133376 module_wrapper.py:139] From bin/story_classifier.py:784: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.


W1205 23:19:25.551558 140157866133376 module_wrapper.py:139] From /gdrive/My Drive/nlp_final/bin/modeling.py:93: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W1205 23:19:25.553454 140157866133376 module_wrapper.py:139] From bin/story_classifier.py:808: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https

In [12]:

!python bin/story_classifier.py --task_name=STORY --do_predict=true --data_dir=tmp/test2 --vocab_file=bin/pre/vocab.txt --bert_config_file=bin/pre/bert_config.json --init_checkpoint=$ckpt_file --max_seq_length=60 --output_dir=tmp/res2





W1205 23:20:23.465477 140224112387968 module_wrapper.py:139] From bin/story_classifier.py:784: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.


W1205 23:20:23.465677 140224112387968 module_wrapper.py:139] From bin/story_classifier.py:784: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.


W1205 23:20:23.466035 140224112387968 module_wrapper.py:139] From /gdrive/My Drive/nlp_final/bin/modeling.py:93: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W1205 23:20:23.467685 140224112387968 module_wrapper.py:139] From bin/story_classifier.py:808: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https

In [0]:
res1 = pd.read_csv("tmp/res1/test_results.tsv",sep="\t",header=None)
res2 = pd.read_csv("tmp/res2/test_results.tsv",sep="\t",header=None)

In [16]:
df_out = test1[["InputStoryid"]]
df_out["score1"] = res1[1]
df_out["score2"] = res2[1]
scores = df_out[["score1","score2"]].values
pred = np.argmax(scores,1)+1
df_out["pred"] = pred
#acc = np.mean(df_out["pred"].values == df_out["AnswerRightEnding"].values)
#print(acc)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [19]:
#save df_out as output
df_out

,InputStoryid,score1,score2,pred
0,9a26ef1d-19a5-4f8b-bd04-7674f598df91,0.624881,0.624469,1
1,2caa6eda-0fb4-4ea1-81b4-d271dd8b6b29,0.628321,0.627706,1
2,f994b657-f673-4843-8145-2244ba93a28d,0.625143,0.625072,1
3,c85e9c3e-4c5a-4b53-8572-52f98228df2c,0.626850,0.628611,2
4,976a11ff-1a3c-4f4f-9b21-3179f2d9cf20,0.633015,0.597097,1
...,...,...,...,...
1866,fa5b1ef3-49f9-48fa-ad34-0d27fa665d37,0.428858,0.582524,2
1867,d42984d5-572a-4286-bea7-22f80865c8b0,0.622762,0.626606,2
1868,b5744737-3ac9-4cf4-a3fd-054a1e6956b2,0.627923,0.627239,1
1869,1dbd0d76-5cb4-4edf-8eb6-b8ccd87b454a,0.625942,0.627336,2
